# 4.3 事前学習: rinnaの事前学習済みGPT-2モデルで継続学習

このノートブックは、4.1節の前処理（正規化→連結→チャンク化）を適用したうえで、
rinnaの事前学習済み日本語GPT-2モデル (`rinna/japanese-gpt2-medium`) を用いて
Causal Language Modeling の継続学習を行います。

- データ: `globis-university/aozorabunko-clean`（train split）
- トークナイザ: rinnaの事前学習済みトークナイザー
- モデル: rinna/japanese-gpt2-medium（事前学習済み）

注意: 大規模学習には時間とGPUが必要です。まずは小さな `max_steps` で動作確認してから、
徐々にスケールさせてください。

In [ ]:
import os

# 使用するGPUを制限（カンマ区切りで複数指定可能）
# 例: "0" → GPU 0のみ使用, "0,1" → GPU 0と1を使用
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


## インポートと設定

In [ ]:
import re
from pathlib import Path
from typing import Iterable, Optional

import torch
from datasets import load_dataset, Dataset

try:
    import neologdn  # 日本語用正規化（任意）
except Exception:
    neologdn = None

from transformers import (
    AutoTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

torch.manual_seed(42)
TEXT_COL = "text"
SEP = "\n\n<|doc|>\n\n"

# rinnaモデルの指定
model_name = "rinna/japanese-gpt2-medium"

# パラメータ（必要に応じて変更）
block_size = 512            # rinnaモデルのmax_position_embeddingsに合わせて調整可能
train_split = 'train'
eval_ratio = 0.01
per_device_train_batch_size = 16
gradient_accumulation_steps = 8
learning_rate = 5e-5        # 事前学習済みモデルなので学習率を下げる
weight_decay = 0.1
warmup_steps = 100
num_train_epochs = 3        # 10エポック学習
logging_steps = None         # 後で計算（0.5エポックごと）
eval_steps = None            # 後で計算（0.5エポックごと）

# 保存先（ノートブック相対パス -> リポジトリ直下に配置）
REPO_ROOT = Path.cwd().parent.parent
output_dir = REPO_ROOT / 'models' / 'rinna-gpt2-aozora-finetuned'
output_dir.mkdir(parents=True, exist_ok=True)
str(output_dir)

## セクション01の前処理済みデータを利用（連結→チャンク化のみ）

In [ ]:
# セクション01の成果物（notebooks/chapter04/data）を読み込み、連結→チャンク化のみ実施
from pathlib import Path
import json

def chunk_text(s: str, size: int) -> Iterable[str]:
    for i in range(0, len(s), size):
        yield s[i : i + size]

# notebooks/chapter04/ からの相対パス
DATA_ROOT = Path('data')
CANDIDATES = [DATA_ROOT / 'aozora', DATA_ROOT]  # 優先順に探す

def load_docs(base: Path, split: str) -> list[str]:
    jsonl = base / f'{split}.jsonl'
    txt   = base / f'{split}.txt'
    if jsonl.exists():
        with jsonl.open('r', encoding='utf-8') as f:
            return [json.loads(line)['text'] for line in f if line.strip()]
    if txt.exists():
        raw = txt.read_text(encoding='utf-8')
        return [s.strip() for s in raw.split('\n\n') if s.strip()]
    return []

train_docs, val_docs = [], []
for base in CANDIDATES:
    if not train_docs:
        train_docs = load_docs(base, 'train')
    if not val_docs:
        val_docs = load_docs(base, 'val')

if not train_docs or not val_docs:
    raise FileNotFoundError('前処理済みデータが見つかりません。notebooks/chapter04/data/(aozora)/{train,val}.{jsonl,txt} を用意してください。')

# 文書をセパレータで連結し、block_size 文字ごとにチャンク
train_long = SEP.join(train_docs)
val_long   = SEP.join(val_docs)
train_chunks = list(chunk_text(train_long, block_size))
val_chunks   = list(chunk_text(val_long,   block_size))

# Hugging Face Datasets へ
train_text_ds = Dataset.from_dict({TEXT_COL: train_chunks})
eval_text_ds  = Dataset.from_dict({TEXT_COL: val_chunks})
len(train_text_ds), len(eval_text_ds)


## rinnaの事前学習済みトークナイザーのロード

In [ ]:
# rinnaの事前学習済みトークナイザーをロード
hf_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# pad_tokenが未設定の場合はeos_tokenを使用
if hf_tokenizer.pad_token is None:
    hf_tokenizer.pad_token = hf_tokenizer.eos_token

print(f"語彙サイズ: {len(hf_tokenizer)}")
print(f"BOS token: {hf_tokenizer.bos_token}")
print(f"EOS token: {hf_tokenizer.eos_token}")
print(f"PAD token: {hf_tokenizer.pad_token}")

## データセットのトークン化

In [ ]:
def tokenize_function(examples):
    return hf_tokenizer(
        examples[TEXT_COL],
        truncation=True,
        max_length=block_size,
    )

tokenized_train = train_text_ds.map(tokenize_function, batched=True, remove_columns=[TEXT_COL])
tokenized_eval = eval_text_ds.map(tokenize_function, batched=True, remove_columns=[TEXT_COL])
tokenized_train[0].keys()

## rinnaの事前学習済みGPT-2モデルで継続学習（Causal LM）

In [ ]:
# rinnaの事前学習済みモデルとコンフィグをロード
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config)

# pad_token_idを設定
model.config.pad_token_id = hf_tokenizer.pad_token_id

print(f"モデル: {model_name}")
print(f"語彙サイズ: {config.vocab_size}")
print(f"最大シーケンス長: {config.n_positions}")
print(f"レイヤー数: {config.n_layer}")
print(f"隠れ層次元: {config.n_embd}")

# データコレーターの設定
data_collator = DataCollatorForLanguageModeling(tokenizer=hf_tokenizer, mlm=False)

# 1エポックあたりのステップ数を計算
train_dataset_size = len(tokenized_train)
steps_per_epoch = train_dataset_size // (per_device_train_batch_size * gradient_accumulation_steps)
print(f"\nデータセットサイズ: {train_dataset_size}")
print(f"1エポックあたりのステップ数: {steps_per_epoch}")

# 0.5エポックごとにログ・評価・保存
logging_steps = max(1, steps_per_epoch // 10)
eval_steps = logging_steps
save_steps = logging_steps
print(f"ログ・評価・保存間隔: {logging_steps} steps (0.5エポックごと)")

# トレーニング引数の設定
training_args = TrainingArguments(
    output_dir=str(output_dir),
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps,
    logging_steps=logging_steps,
    eval_strategy='steps',
    eval_steps=eval_steps,
    save_steps=save_steps,
    save_total_limit=3,
    report_to=['none'],
    load_best_model_at_end=False,
)

# テスト用プロンプト
test_prompts = [
    '吾輩は猫である。名前はまだ無い。',
    '明治時代の',
    '東京の街には',
    '先生は言った。「',
]

# カスタムTrainerクラス（定期的にテキスト生成を実行）
from transformers import TrainerCallback
import logging

class GenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, test_prompts, generation_interval):
        self.tokenizer = tokenizer
        self.test_prompts = test_prompts
        self.generation_interval = generation_interval
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.has_generated_initial = False
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        file_handler = logging.FileHandler('training_generation.log')
        file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        self.logger.addHandler(file_handler)
    
    def on_step_begin(self, args, state, control, model=None, **kwargs):
        # 最初のステップで初期生成を実行
        if not self.has_generated_initial:
            self._generate_samples(model, 0)
            self.has_generated_initial = True
    
    def on_log(self, args, state, control, model=None, **kwargs):
        # generation_intervalごとに生成テストを実行
        current_step = state.global_step
        if current_step > 0 and current_step % self.generation_interval == 0:
            self._generate_samples(model, current_step)
    
    def _generate_samples(self, model, step):
        """テストプロンプトで生成サンプルを表示"""
        self.logger.info(f"\n{'='*60}")
        self.logger.info(f"Step {step}: テキスト生成サンプル")
        self.logger.info(f"{'='*60}")
        
        # モデルの状態を保存
        was_training = model.training
        model.eval()
        
        for prompt in self.test_prompts:
            self.logger.info(f"\nプロンプト: {prompt}")
            self.logger.info("-" * 50)
            
            inputs = self.tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                out = model.generate(
                    **inputs,
                    max_new_tokens=80,
                    do_sample=True,
                    temperature=0.8,
                    top_p=0.9,
                    repetition_penalty=1.2,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            
            generated_text = self.tokenizer.decode(out[0], skip_special_tokens=True)
            self.logger.info(generated_text)
        
        self.logger.info(f"\n{'='*60}\n")
        
        # モデルの状態を復元
        if was_training:
            model.train()

# コールバックの作成（0.5エポックごとに生成）
generation_callback = GenerationCallback(
    tokenizer=hf_tokenizer,
    test_prompts=test_prompts,
    generation_interval=logging_steps
)

# トレーナーの作成
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=hf_tokenizer,
    callbacks=[generation_callback],
)

# トレーニング実行
trainer.train()

# モデルとトークナイザーの保存
trainer.save_model(str(output_dir))
hf_tokenizer.save_pretrained(str(output_dir))
print(f'モデルを保存しました: {output_dir}')

In [ ]:
# 直近の学習ログを表示（必要に応じて調整）
import pandas as pd
pd.DataFrame(trainer.state.log_history).tail(20)


## 簡単な生成テスト（任意）

In [ ]:
# 学習済みモデルでの生成テスト
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM

# 最新のチェックポイントまたは最終保存モデルをロード
# チェックポイントがある場合はそちらを使用、なければ output_dir を使用
checkpoint_dirs = [d for d in output_dir.iterdir() if d.is_dir() and d.name.startswith('checkpoint-')]
if checkpoint_dirs:
    # 最新のチェックポイントを選択（番号順でソート）
    latest_checkpoint = sorted(checkpoint_dirs, key=lambda x: int(x.name.split('-')[1]))[-1]
    model_dir = latest_checkpoint
    print(f"チェックポイントからロード: {model_dir}")
else:
    model_dir = output_dir
    print(f"最終モデルからロード: {model_dir}")

tokenizer = AutoTokenizer.from_pretrained(str(model_dir))
model = AutoModelForCausalLM.from_pretrained(str(model_dir))

# pad_token が未設定の場合は eos を代用
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

# デバイスの設定
device = 'cuda' if torch.cuda.is_available() else ('mps' if getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available() else 'cpu')
model = model.to(device).eval()
print(f"デバイス: {device}")

# 生成テスト
prompts = [
    '吾輩は猫である。名前はまだ無い。',
    '明治時代の',
    '東京の街には',
    '先生は言った。「',
]
for prompt in prompts:
    print(f"\nプロンプト: {prompt}")
    print("-" * 50)

    inputs = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=False,
            temperature=0.8,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # 生成されたテキスト全体を表示
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)
    print(generated_text)

In [ ]:
# 複数のプロンプトでテスト
test_prompts = [
    '吾輩は猫である。名前はまだ無い。',
    '明治時代の',
    '東京の街には',
    '先生は言った。「',
]

for prompt in test_prompts:
    print(f"\nプロンプト: {prompt}")
    print("-" * 50)
    
    inputs = tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)
    print(generated_text)
    print()

## 比較: Zero-shot（学習前の元のrinnaモデル）での出力

In [ ]:
# 元のrinnaモデルをロードしてzero-shotでの生成を試す
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

print("=" * 60)
print("Zero-shot: 学習前の元のrinnaモデルでの生成")
print("=" * 60)

# 元のrinnaモデルをロード
zeroshot_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
zeroshot_model = AutoModelForCausalLM.from_pretrained(model_name)

# pad_tokenが未設定の場合はeos_tokenを使用
if zeroshot_tokenizer.pad_token is None:
    zeroshot_tokenizer.pad_token = zeroshot_tokenizer.eos_token
    zeroshot_model.config.pad_token_id = zeroshot_tokenizer.pad_token_id

# デバイスの設定
device = 'cuda' if torch.cuda.is_available() else ('mps' if getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available() else 'cpu')
zeroshot_model = zeroshot_model.to(device).eval()
print(f"デバイス: {device}\n")

# 同じプロンプトでテスト
test_prompts = [
    '吾輩は猫である。名前はまだ無い。',
    '明治時代の',
    '東京の街には',
    '先生は言った。「',
]

for prompt in test_prompts:
    print(f"\nプロンプト: {prompt}")
    print("-" * 50)
    
    inputs = zeroshot_tokenizer(prompt, return_tensors='pt', add_special_tokens=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        out = zeroshot_model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            pad_token_id=zeroshot_tokenizer.pad_token_id,
            eos_token_id=zeroshot_tokenizer.eos_token_id,
        )
    
    generated_text = zeroshot_tokenizer.decode(out[0], skip_special_tokens=True)
    print(generated_text)
    print()